In [54]:
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('dark_background')
print("Setup Complete")

Setup Complete


In [55]:
train_path = "../input/thapar-summer-school-basic-classification/Train_dataset.csv"
test_path = "../input/thapar-summer-school-basic-classification/test_dataset.csv"

train_data = pd.read_csv(train_path,index_col="patient_id")
test_data = pd.read_csv(test_path,index_col="patient_id")

train_data.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
patient_id,,,,,,,,,,,,,,
179,57,1,0,150,276,0,0,112,1,0.6,1,1,1,0
228,59,1,3,170,288,0,0,159,0,0.2,1,0,3,0
111,57,1,2,150,126,1,1,173,0,0.2,2,1,3,1
246,56,0,0,134,409,0,0,150,1,1.9,1,2,3,0
60,71,0,2,110,265,1,0,130,0,0.0,2,1,2,1


In [56]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 242 entries, 179 to 47
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       242 non-null    int64  
 1   sex       242 non-null    int64  
 2   cp        242 non-null    int64  
 3   trtbps    242 non-null    int64  
 4   chol      242 non-null    int64  
 5   fbs       242 non-null    int64  
 6   restecg   242 non-null    int64  
 7   thalachh  242 non-null    int64  
 8   exng      242 non-null    int64  
 9   oldpeak   242 non-null    float64
 10  slp       242 non-null    int64  
 11  caa       242 non-null    int64  
 12  thall     242 non-null    int64  
 13  output    242 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 28.4 KB


In [57]:
from catboost import CatBoostClassifier, Pool
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

In [58]:
cols = [i for i in train_data.columns if i!="output"]
seed = np.random.seed(0)

X = train_data[cols]
y = train_data["output"]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3,random_state=seed)

In [59]:
rfmodel = RandomForestClassifier(random_state=seed)
rfmodel.fit(X_train,y_train)
print("-"*100)
print("Accuracy of RF:",accuracy_score(y_test,rfmodel.predict(X_test)))
print("Cross Val Accuracy:",np.mean(cross_val_score(rfmodel,X,y,cv=5,scoring='accuracy')))

----------------------------------------------------------------------------------------------------
Accuracy of RF: 0.7671232876712328
Cross Val Accuracy: 0.8179421768707483


In [60]:
gbmodel = GradientBoostingClassifier(random_state=seed)
gbmodel.fit(X_train,y_train)
print("-"*100)
print("Accuracy of GB:",accuracy_score(y_test,gbmodel.predict(X_test)))
print("Cross Val Accuracy:",np.mean(cross_val_score(gbmodel,X,y,cv=5,scoring='accuracy')))

----------------------------------------------------------------------------------------------------
Accuracy of GB: 0.7397260273972602
Cross Val Accuracy: 0.7852040816326531


In [61]:
xgbmodel = XGBClassifier(random_state=seed)
xgbmodel.fit(X_train,y_train)
print("-"*100)
print("Accuracy of XGB:",accuracy_score(y_test,xgbmodel.predict(X_test)))
print("Cross Val Accuracy:",np.mean(cross_val_score(xgbmodel,X,y,cv=5,scoring='accuracy')))

----------------------------------------------------------------------------------------------------
Accuracy of XGB: 0.8082191780821918
Cross Val Accuracy: 0.7974489795918368


In [62]:
lgbmmodel = LGBMClassifier(random_state=seed)
lgbmmodel.fit(X_train,y_train)
print("-"*100)
print("Accuracy of LGBM:",accuracy_score(y_test,lgbmmodel.predict(X_test)))
print("Cross Val Accuracy:",np.mean(cross_val_score(lgbmmodel,X,y,cv=5,scoring='accuracy')))

----------------------------------------------------------------------------------------------------
Accuracy of LGBM: 0.8356164383561644
Cross Val Accuracy: 0.8095238095238095


In [63]:
catmodel = CatBoostClassifier(random_state=seed,verbose=False)
catmodel.fit(X_train,y_train)
print("-"*100)
print("Accuracy of CAT:",accuracy_score(y_test,catmodel.predict(X_test)))
print("Cross Val Accuracy:",np.mean(cross_val_score(catmodel,X,y,cv=5,scoring='accuracy')))

----------------------------------------------------------------------------------------------------
Accuracy of CAT: 0.7808219178082192
Cross Val Accuracy: 0.8056972789115647


In [64]:
vcmodel = VotingClassifier([("rf",rfmodel),("gb",gbmodel),("xgb",xgbmodel),("lgbm",lgbmmodel),("cat",catmodel)])
vcmodel.fit(X_train,y_train)
print("-"*100)
print("Accuracy of VC:",accuracy_score(y_test,vcmodel.predict(X_test)))
print("Cross Val Accuracy:",np.mean(cross_val_score(vcmodel,X,y,cv=5,scoring='accuracy')))

----------------------------------------------------------------------------------------------------
Accuracy of VC: 0.7945205479452054
Cross Val Accuracy: 0.8221088435374149


# Making Submission Files

In [65]:
submission = test_data.copy()

colsToDrop = [i for i in test_data.columns if i!="patient_id" and i!="output"]
submission.drop(colsToDrop,inplace=True,axis=1)

In [66]:
models = {
    "rf":rfmodel,
    "gb":gbmodel,
    "xgb":xgbmodel,
    "lgbm":lgbmmodel,
    "cat":catmodel,
    "vc":vcmodel,
}

In [67]:
for model in models:
    models[model].fit(X,y)
    y_pred = models[model].predict(test_data)
    submission["output"] = y_pred
    filename = f"submission_{model}.csv"
    submission.to_csv(filename,index=True,header=True)